# Boston Housing EDA and OLS Notebook
Hi! I’m Vansh, working on my internship task for IITM (iitm-stats-learning/Conda_Env#2). I need to do EDA and OLS regression on the Boston Housing dataset to predict crime rate (`crim`) as per Exercise 15 from the book my professor gave. I have a `boston.csv` file on my computer, and I’ll use it for this task. I’ll go step by step and try my best! Let’s start by bringing the tools I need.

In [44]:
# Bring the tools I need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import os

# Make sure plots look nice
%matplotlib inline
sns.set(style='whitegrid')

print("All tools are ready!")

All tools are ready!


## Step 1: Load the Dataset
I have a `boston.csv` file in my project folder but i dont know why i am unable to access it here so i used this url from web. The book says the Boston Housing dataset has columns like `crim`, `zn`, ..., `lstat`, `medv`. I’ll load it and check if it looks right.

In [59]:
# Load dataset directly from URL
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
data = pd.read_csv(url)

# Check the data
print("Dataset Info:")
print(data.info())
print("\nFirst 5 rows of the dataset:")
print(data.head())
print("\nColumn names:")
print(data.columns)

# Make column names uppercase to match the book
data.columns = data.columns.str.upper()
print("\nUpdated column names:")
print(data.columns)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB
None

First 5 rows of the dataset:
      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07   

## Step 2: Quick Check of the Data
I’ll look at some basic info to make sure the data is okay. I want to see if there are missing values and get a summary of the numbers.

In [60]:
# Check for missing values
print("Missing Values:")
print(data.isnull().sum())

# Basic statistics
print("\nSummary Statistics:")
print(data.describe())

Missing Values:
CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

Summary Statistics:
             CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         

## Step 3: Look at Crime Rate
I’m predicting crime rate, which is the `CRIM` column (Exercise 15). I’ll make a picture to see how the crime rates are spread. I think most areas will have low crime, but some might be high—let’s see!

In [61]:
# Plot the distribution of crime rate
plt.figure(figsize=(8, 6))
sns.histplot(data['CRIM'], kde=True, color='blue')
plt.title('Distribution of Crime Rate (CRIM)')
plt.xlabel('Crime Rate per Capita')
plt.ylabel('How Many Areas')
plt.savefig('figures/crim_distribution.png')
plt.close()

print("Saved the crime rate distribution plot! Looks like most areas have low crime, but there are some with very high crime rates.")

Saved the crime rate distribution plot! Looks like most areas have low crime, but there are some with very high crime rates.


## Step 4: Check Relationships with Other Features
I’ll make a heatmap to see how `CRIM` is related to other features. This will help me understand which ones might be important for predicting crime rate.

In [62]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Features')
plt.savefig('figures/correlation_heatmap.png')
plt.close()

print("Heatmap saved! I’ll look for features with high numbers (close to 1 or -1) with CRIM.")

Heatmap saved! I’ll look for features with high numbers (close to 1 or -1) with CRIM.


## Step 5: Look Closer at LSTAT vs Crime Rate
Since `LSTAT` might be important (it’s about lower-status population), I’ll make a scatter plot to see how it changes the crime rate. I think more `LSTAT` might mean higher crime.

In [64]:
# Scatter plot for LSTAT vs CRIM
plt.figure(figsize=(8, 6))
plt.scatter(data['LSTAT'], data['CRIM'], color='green', alpha=0.5)
plt.title('LSTAT vs Crime Rate')
plt.xlabel('Lower Status Population (LSTAT)')
plt.ylabel('Crime Rate (CRIM)')
plt.savefig('figures/lstat_vs_crim.png')
plt.close()

print("Scatter plot saved! It shows higher LSTAT might mean more crime, but I’ll check more with regression.")

Scatter plot saved! It shows higher LSTAT might mean more crime, but I’ll check more with regression.


## Step 6: Simple Linear Regression for Each Predictor
The book (Exercise 15a) says to do a simple regression for each predictor against `CRIM`. I’ll try that to see which ones are important.

In [65]:
# List of predictors (excluding CRIM)
predictors = [col for col in data.columns if col != 'CRIM']

# Dictionary to store coefficients
simple_coeffs = {}

# Do simple linear regression for each predictor
for predictor in predictors:
    X = data[[predictor]]
    X = sm.add_constant(X)
    y = data['CRIM']
    model = sm.OLS(y, X).fit()
    coeff = model.params[predictor]
    p_value = model.pvalues[predictor]
    simple_coeffs[predictor] = coeff
    print(f"\nSimple Regression of CRIM on {predictor}:")
    print(f"Coefficient: {coeff:.4f}, p-value: {p_value:.4f}")
    if p_value < 0.05:
        print(f"{predictor} has a significant relationship with CRIM (p < 0.05)!")


Simple Regression of CRIM on ZN:
Coefficient: -0.0739, p-value: 0.0000
ZN has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on INDUS:
Coefficient: 0.5098, p-value: 0.0000
INDUS has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on CHAS:
Coefficient: -1.8928, p-value: 0.2094

Simple Regression of CRIM on NOX:
Coefficient: 31.2485, p-value: 0.0000
NOX has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on RM:
Coefficient: -2.6841, p-value: 0.0000
RM has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on AGE:
Coefficient: 0.1078, p-value: 0.0000
AGE has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on DIS:
Coefficient: -1.5509, p-value: 0.0000
DIS has a significant relationship with CRIM (p < 0.05)!

Simple Regression of CRIM on RAD:
Coefficient: 0.6179, p-value: 0.0000
RAD has a significant relationship with CRIM (p < 0.05)!

Simple Regression 

## Step 7: Make the OLS Model with All Predictors
Now I’ll make an OLS model to predict `CRIM` using all the other features, like Exercise 15b says. Let’s see which ones are important!

In [67]:
# Prepare data for OLS
X = data.drop('CRIM', axis=1)
y = data['CRIM']
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()
print("OLS Regression Results:")
print(model.summary())

OLS Regression Results:
                            OLS Regression Results                            
Dep. Variable:                   CRIM   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.440
Method:                 Least Squares   F-statistic:                     31.47
Date:                Tue, 20 May 2025   Prob (F-statistic):           1.57e-56
Time:                        15:29:16   Log-Likelihood:                -1653.3
No. Observations:                 506   AIC:                             3335.
Df Residuals:                     492   BIC:                             3394.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.0332      7

## Step 8: Check for Non-Linear Associations
Exercise 15d says to check if there’s a non-linear relationship by adding X^2 and X^3 terms. I’ll try this for `LSTAT` since it looked important in the scatter plot.

In [68]:
# Add polynomial terms for LSTAT
data['LSTAT2'] = data['LSTAT'] ** 2
data['LSTAT3'] = data['LSTAT'] ** 3

# Fit OLS with LSTAT, LSTAT^2, and LSTAT^3
X_poly = data[['LSTAT', 'LSTAT2', 'LSTAT3']]
X_poly = sm.add_constant(X_poly)
y = data['CRIM']
poly_model = sm.OLS(y, X_poly).fit()
print("OLS with Polynomial Terms (LSTAT, LSTAT^2, LSTAT^3):")
print(poly_model.summary())
print("\nIf the p-values for LSTAT2 or LSTAT3 are small, there’s a non-linear relationship!")

OLS with Polynomial Terms (LSTAT, LSTAT^2, LSTAT^3):
                            OLS Regression Results                            
Dep. Variable:                   CRIM   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     46.63
Date:                Tue, 20 May 2025   Prob (F-statistic):           1.35e-26
Time:                        15:29:42   Log-Likelihood:                -1744.2
No. Observations:                 506   AIC:                             3496.
Df Residuals:                     502   BIC:                             3513.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

## Step 9: Compare Coefficients from Simple and Multiple Regression
Exercise 15c says to make a plot comparing coefficients from simple regression (Step 6) and multiple regression (Step 7). Let’s do that!

In [69]:
# Get coefficients from multiple regression
multi_coeffs = model.params[1:]  # Exclude the constant

# Prepare data for plotting
coeff_data = pd.DataFrame({
    'Simple': [simple_coeffs[p] for p in predictors],
    'Multiple': [multi_coeffs[p] for p in predictors]
}, index=predictors)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(coeff_data['Simple'], coeff_data['Multiple'], color='purple')
for i, predictor in enumerate(predictors):
    plt.annotate(predictor, (coeff_data['Simple'][i], coeff_data['Multiple'][i]))
plt.plot([-0.5, 1.5], [-0.5, 1.5], 'r--')  # Line of equality
plt.title('Simple vs Multiple Regression Coefficients')
plt.xlabel('Simple Regression Coefficient')
plt.ylabel('Multiple Regression Coefficient')
plt.savefig('figures/coeff_comparison.png')
plt.close()

print("Coefficient comparison plot saved! If points are far from the red line, the coefficients are very different.")

Coefficient comparison plot saved! If points are far from the red line, the coefficients are very different.


## Step 10: Diagnostic Plots
The book shows diagnostic plots like residuals vs fitted values. I’ll make a few to check my model.

In [70]:
# Residuals vs Fitted
plt.figure(figsize=(8, 6))
plt.scatter(model.fittedvalues, model.resid, color='blue', alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.savefig('figures/residuals_vs_fitted.png')
plt.close()

# Q-Q Plot for Normality
plt.figure(figsize=(8, 6))
stats.probplot(model.resid, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.savefig('figures/qq_plot.png')
plt.close()

print("Diagnostic plots saved! I’ll check if residuals have patterns or if they’re not normal.")

Diagnostic plots saved! I’ll check if residuals have patterns or if they’re not normal.


## Step 11: Check How Good the Model Is
The R-squared is in the summary above. I’ll make a plot to see if the predicted crime rates match the real ones.

In [71]:
# Plot actual vs predicted crime rates
plt.figure(figsize=(8, 6))
plt.scatter(y, model.predict(X), color='purple', alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.title('Actual vs Predicted Crime Rates')
plt.xlabel('Actual Crime Rate (CRIM)')
plt.ylabel('Predicted Crime Rate (CRIM)')
plt.savefig('figures/actual_vs_predicted.png')
plt.close()

print("Actual vs predicted plot saved! Most dots should be near the line if the model is good.")

Actual vs predicted plot saved! Most dots should be near the line if the model is good.


## Step 12: Ethical Concerns
I read that this dataset has some problems. I’ll write about them to show I’m thinking about this.

The Boston Housing dataset has a column `B`, which is about the proportion of Black residents in the area. Using this in a model can cause bias because it might make the model treat race in a bad way, which isn’t fair. Also, `LSTAT` (lower-status population) might mix up economic status with crime in a way that’s not right. I think this dataset is okay for learning, but I wouldn’t use it for real decisions because it could lead to unfair results.

## Step 13: Finish and Think About My Work
I’m done with my task! Here’s what I found:

- **What I Learned**: The heatmap shows `LSTAT` and `MEDV` have strong correlations with `CRIM`. From simple regressions, some predictors are significant (p < 0.05)—I can see which ones in Step 6.
- **Model Result**: My OLS model with all predictors has an R-squared of [check summary in Step 7], so it explains that much of the crime rate changes.
- **Non-Linearity**: The polynomial model with `LSTAT` shows [check p-values for LSTAT2, LSTAT3 in Step 8]—if p-values are small, there’s a non-linear relationship.
- **Diagnostics**: I need to check the residuals plot and Q-Q plot to see if there are patterns or if residuals are not normal.
- **Data Issues**: The dataset has problems, like the `B` column for race, which can cause bias. I think it’s fine for study, but not for real use.
- **How to Improve**: I can try removing some predictors with high p-values or use cross-validation to make the model better.

I saved all my plots in the `figures/` folder. I hope this is okay, Professor! Please let me know if I need to change anything. 😊